# Notebook on Initialization of the GurobiModelBuilder

This notebook shows variants of how to import a neural network in ONNX format.
First it is showcased on a compatible network. Then, options are demonstrated if a network is not compatible.

### 1. Initialization of Compatible Network

First, the GurobiModelBuilder class has to be imported.

In [21]:
from enncode.gurobiModelBuilder import GurobiModelBuilder

Then the path to the desired ONNX network is defined.

In [22]:
# Define the path to the desired ONNX network
path = "data/conv_net.onnx"

The path is passed to the constructor as an argument. The <em>GurobiModelBuilder</em> class then parses the given ONNX network in topological order. By calling the subsequent <em>build_model()</em> function, the network is encoded stepwise by applying the constraints to the underlying GurobiModel.

In [23]:
model_builder = GurobiModelBuilder(path)
model_builder.build_model()

Upon successful initialization, the GurobiModel and corresponding input/output variables can be obtained.

In [24]:
gurobi_model = model_builder.get_gurobi_model()
input_vars = model_builder.get_input_vars()
output_vars = model_builder.get_output_vars()

_(Optional)_ If the encoding should be verified, the optional parameter _compcheck_ can be set to True. By doing so, the _build_model()_ function includes a basic _compatibility_check_ of our library. This validation compares the outputs produced by the encoded optimization model with those computed by the ONNX Runtime. Concretely, input variables in the MILO model are fixed to randomly sampled values to define a unique optimization problem; after solving it, the resulting output assignments are compared against reference outputs obtained from ONNX Runtime execution. The rtol and atol arguments define the accepted tolerance between those inputs, which might be caused to floating point arithmetic.

In [25]:
model_builder = GurobiModelBuilder(path, compcheck=True, rtol=1e-3, atol=1e-4)
model_builder.build_model()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 896 rows, 2486 columns and 16012 nonzeros
Model fingerprint: 0x806621ab
Model has 796 simple general constraints
  796 MAX
Variable types: 2486 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [9e-05, 1e+00]
  RHS range        [7e-06, 6e-01]
Presolve removed 896 rows and 2486 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

 Given network has been com

### 2. Non-compatible Network

Assume there is a non-piecewise linear network, e.g. a classifier containing the _SoftMax_ activation. The shown initialization above will raise a _NotImplementedError_ caused by the _SoftMax_ activation.
However, to improve user-friendliness, additional compatibility checks can be used. If the compatibility check fails, the method exits with status code 1.

In [26]:
from enncode.compatibility import compatibility_check
path = "data/conv_softmax.onnx"

try:
    compatibility_check(path, iterative_analysis=False, rtol=1e-3, atol=1e-4)
except SystemExit:
    print("\n If the compatibility check fails, the method exits with status code 1.")


 An error has occurred by solving gurobi model for given onnx file.

 No iterative check for compatibility is performed. 

GurobiModelBuilder misses support for following node types:
['Softmax']

 If the compatibility check fails, the method exits with status code 1.


Unfortunately, the last operation of the network is not supported. However, if desired, the user can check which last subgraph of the network is compatible. This could then be reused, for example, to encode unsupported operations themselves. Therefore, the _iterative_analysis_ flag can be set to True.

In [27]:
try:
    compatibility_check(path, iterative_analysis=True, output_dir="data/softmax_subgraphs/", rtol=1e-3, atol=1e-4)
except SystemExit:
    print("\n Each subgraph was checked for compatibility issues. The results are stored in the directory \"data/softmax_subgraphs/subgraphs\"")


 An error has occurred by solving gurobi model for given onnx file.
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 676 rows, 1798 columns and 11492 nonzeros
Model fingerprint: 0xd306bd69
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [2e-03, 1e+00]
  RHS range        [2e-02, 1e-01]
Presolve removed 676 rows and 1798 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 570

The ONNX file of the last compatible graph can then be used. This corresponds to the second-to-last ONNX file in the directory. Alternatively, it can also be identified by using the _subgraphs_log_ file. This file lists the results of each individual subgraph. This may help identifying nodes that are exhibiting malfunctions.

In [28]:
last_compatible = "data/softmax_subgraphs/subgraphs/node_008__output_Gemm.onnx"
model_builder = GurobiModelBuilder(last_compatible, compcheck=True, rtol=1e-03, atol=1e-04)
model_builder.build_model()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 916 rows, 2506 columns and 16232 nonzeros
Model fingerprint: 0x801873eb
Model has 796 simple general constraints
  796 MAX
Variable types: 2506 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [3e-04, 1e+00]
  RHS range        [4e-03, 3e-01]
Presolve removed 916 rows and 2506 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

 Given network has been com